Look at utterances, by finding the marginal in onehot space, for each meaning type/value combination, then argmaxing it to find some kind of. Well... let's find the entropy of each utterance position, for each type/value combination.

In [ ]:
import torch
import os
from os import path
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import math
import sys
if '..' not in sys.path:
    print('appending .. to path')
    sys.path.append('..')
import ulfs
from ulfs.utils import expand, die
from mll import e2e_fixpoint
from mll import run_mem_recv, run_mem_send

In [ ]:
# model_path = '../models/mll236_230k.dat'
# model_path = '../tmp/e2e_fixpoint_mll241_hughtok2_20190106_195828.dat'
model_path = '../tmp/e2e_fixpoint_mll244_hughtok4_20190107_001304.dat'

with open(model_path, 'rb') as f:
    model_dat = torch.load(f)
print(model_dat.keys())
episode = model_dat['episode']
print('episode', episode)
from ulfs.params import Params
params = model_dat['params']
print('params', params)

default_params = e2e_fixpoint.default_params
for k, v in default_params.items():
    if k not in params:
        setattr(params, k, v)

sender_agent = run_mem_send.Agent(params)
print('sender_agent', sender_agent)
sender_model = sender_agent.model
print('sender_model', sender_model)
sender_model.load_state_dict(model_dat['sender_model_state'])
print('sender_model', sender_model)

p = params
tot_examples = int(math.pow(p.meanings_per_type, p.num_meaning_types))
print('tot_examples', tot_examples)
meanings_ints = torch.from_numpy(np.arange(tot_examples))
# meanings = torch.zeros(*([p.meanings_per_type] * p.num_meaning_types))
meanings = torch.zeros(tot_examples, p.num_meaning_types, dtype=torch.int64)
print('meanings.size()', meanings.size())
for t in range(p.num_meaning_types):
    meanings[:, p.num_meaning_types - t - 1] = meanings_ints % p.meanings_per_type
    meanings_ints = meanings_ints // p.meanings_per_type
# print('meanings[:110]', meanings[:110])

# sample_idxes = torch.from_numpy(np.random.choice(tot_examples, 1000, replace=False))
# # meanings = meanings[:1000]
# meanings = meanings[sample_idxes]
# print('meanings.size()', meanings.size())

utt_logits = sender_model(meanings)
# print('utt_logits', utt_logits)
print('utt_logits.size()', utt_logits.size())
_, utt_preds = utt_logits.max(dim=-1)
print('utt_preds.size()', utt_preds.size())
# for n in range(110):
#     print(utt_preds[:, n])
# target_meaning = []
# print('utt_preds[:, :1000]', utt_preds[:, :1000])
target_meaning_type = 0
target_meaning_value = 0
target_meaning_mask = meanings[:, target_meaning_type] == target_meaning_value
target_meaning_idxes = target_meaning_mask.nonzero().view(-1).long()
# print('target_meaning_idxes', target_meaning_idxes)
print('target_meaning_idxes.size()', target_meaning_idxes.size())
# for i, v in enumerate(target_meaning_idxes.tolist()):
#     print(''.join([str(v) for v in utt_preds[:, v].tolist()]))
# print('utt_preds[:, target_meaning_idxes]', utt_preds[:, target_meaning_idxes])

utt_len = utt_preds.size(0)
vocab_size = params.vocab_size
print('utt_len', utt_len, 'vocab_size', vocab_size)
N = utt_preds.size(1)
print('N', N)
entropies = []
for t in range(utt_len):
    entropy = 0
    for i in range(vocab_size):
        p_i = (utt_preds[t] == i).float().sum().item() / N
        if p_i > 0:
            entropy -= p_i * math.log(p_i)
    entropies.append(entropy)
print(' '.join(['%.2f' % e for e in entropies]))
#     print(t, '%.3f' % entropy)